# Random Forest Regressor

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
df=pd.read_csv("./cardekho_imputated.csv",index_col=[0])
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


# Data Cleaning

Handling Missing Values

In [4]:
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [5]:
df.drop(['car_name','brand'],axis=1,inplace=True)

In [7]:
df.model.unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [11]:
## Get all Different Types of Features
num_features=[feature for feature in df.columns if df[feature].dtype!="O"]
print("Number of Numerical Features ",len(num_features))
cat_features=[feature for feature in df.columns if df[feature].dtype=="O"]
print("Number of Categorical Features ",len(cat_features))
discrete_featuers=[feature for feature in num_features if len(df[feature].unique())<=25]
print("Number of Discrete Features ",len(discrete_featuers))
continuous_features=[feature for feature in num_features if feature not in discrete_featuers]
print("Number of Continuous Features ",len(continuous_features))

Number of Numerical Features  7
Number of Categorical Features  4
Number of Discrete Features  2
Number of Continuous Features  5


In [12]:
from sklearn.model_selection import train_test_split
X=df.drop(['selling_price'],axis=1)
y=df['selling_price']

In [13]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


# Feature Encoding and Scaling

In [14]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['model']=le.fit_transform(X['model'])

In [15]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [16]:
# Create column transformer with 3 types of transformers
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
num_features=X.select_dtypes(exclude="object").columns
onehot_columns=["seller_type","fuel_type","transmission_type"]
numeric_transfoerm=StandardScaler()
oh_transformer=OneHotEncoder(drop="first")
preprocessor=ColumnTransformer(
    [
        ('OneHotEncoder', oh_transformer,onehot_columns),
        ('StandardScaler',numeric_transfoerm,num_features)
    ],remainder='passthrough'
)

In [17]:
X=preprocessor.fit_transform(X)

In [19]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [21]:
X_train.shape,X_test.shape

((12328, 14), (3083, 14))

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [27]:
# Create function to evaluate model
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2=r2_score(true,predicted)
    return mae,mse,rmse,r2

In [28]:
models={
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor()
}

for i in range(len(list(models))):
    # Model Training
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    # Prediction
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    # Train evaluation
    model_train_mae,model_train_mse,model_train_rmse,model_train_r2_score=evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_mse,model_test_rmse,model_test_r2_score=evaluate_model(y_test,y_test_pred)
    print(list(models.keys())[i])
    print('Model performance for Training set')
    print(f'- Root Mean Squared Error: {model_train_rmse:.4f}')
    print(f'- Mean Squared Error: {model_train_mse:.4f}')
    print(f'- Mean Absolute Error: {model_train_mae:.4f}')
    print(f'- R2 Score: {model_train_r2_score:.4f}')
    print('-'*35)
    print('Model performance for Test set')
    print(f'- Root Mean Squared Error: {model_test_rmse:.4f}')
    print(f'- Mean Squared Error: {model_test_mse:.4f}')
    print(f'- Mean Absolute Error: {model_test_mae:.4f}')
    print(f'- R2 Score: {model_test_r2_score:.4f}')
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 553855.6665
- Mean Squared Error: 306756099359.7596
- Mean Absolute Error: 268101.6071
- R2 Score: 0.6218
-----------------------------------
Model performance for Test set
- Root Mean Squared Error: 502543.5930
- Mean Squared Error: 252550062888.5656
- Mean Absolute Error: 279618.5794
- R2 Score: 0.6645


Lasso
Model performance for Training set
- Root Mean Squared Error: 553855.6710
- Mean Squared Error: 306756104248.3742
- Mean Absolute Error: 268099.2226
- R2 Score: 0.6218
-----------------------------------
Model performance for Test set
- Root Mean Squared Error: 502542.6696
- Mean Squared Error: 252549134806.7813
- Mean Absolute Error: 279614.7461
- R2 Score: 0.6645


Ridge
Model performance for Training set
- Root Mean Squared Error: 553856.3160
- Mean Squared Error: 306756818740.9266
- Mean Absolute Error: 268059.8015
- R2 Score: 0.6218
-----------------------------------
Model performance for Test

In [29]:
knn_params={'n_neighbors':[2,3,10,20,40,50]}
rf_params={'max_depth':[5,8,15,None,10],
           'max_features':[5,7,"auto",8],
           'min_samples_split':[2,8,15,20],
           'n_estimators':[100,200,500,1000]}

In [30]:
randomcv_models=[
    ('KNN',KNeighborsRegressor(),knn_params),
    ('RF',RandomForestRegressor(),rf_params)
]

In [31]:
# Hyper parameter tuning
from sklearn.model_selection import RandomizedSearchCV
model_param={}
for name,model,params in randomcv_models:
    random=RandomizedSearchCV(estimator=model,
                              param_distributions=params,
                              n_iter=100,
                              cv=3,
                              verbose=2,
                              n_jobs=-1)
    random.fit(X_train,y_train)
    model_param[name]=random.best_params_
for model_name in model_param:
    print(f"---------Best Params for {model_name}-----------")
    print(model_param[model_name])

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
---------Best Params for KNN-----------
{'n_neighbors': 10}
---------Best Params for RF-----------
{'n_estimators': 200, 'min_samples_split': 2, 'max_features': 7, 'max_depth': None}


In [32]:
models={
    "Random Forest Regressor": RandomForestRegressor(n_estimators=200,min_samples_split=2,max_features=7,max_depth=None),
    "K Nearest Neighbor": KNeighborsRegressor(n_neighbors=10)
}
for i in range(len(list(models))):
    # Model Training
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    # Prediction
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    # Train evaluation
    model_train_mae,model_train_mse,model_train_rmse,model_train_r2_score=evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_mse,model_test_rmse,model_test_r2_score=evaluate_model(y_test,y_test_pred)
    print(list(models.keys())[i])
    print('Model performance for Training set')
    print(f'- Root Mean Squared Error: {model_train_rmse:.4f}')
    print(f'- Mean Squared Error: {model_train_mse:.4f}')
    print(f'- Mean Absolute Error: {model_train_mae:.4f}')
    print(f'- R2 Score: {model_train_r2_score:.4f}')
    print('-'*35)
    print('Model performance for Test set')
    print(f'- Root Mean Squared Error: {model_test_rmse:.4f}')
    print(f'- Mean Squared Error: {model_test_mse:.4f}')
    print(f'- Mean Absolute Error: {model_test_mae:.4f}')
    print(f'- R2 Score: {model_test_r2_score:.4f}')
    print('='*35)
    print('\n')

Random Forest Regressor
Model performance for Training set
- Root Mean Squared Error: 120623.9800
- Mean Squared Error: 14550144555.4708
- Mean Absolute Error: 38875.8605
- R2 Score: 0.9821
-----------------------------------
Model performance for Test set
- Root Mean Squared Error: 214743.7407
- Mean Squared Error: 46114874166.5788
- Mean Absolute Error: 98682.1074
- R2 Score: 0.9387


K Nearest Neighbor
Model performance for Training set
- Root Mean Squared Error: 363460.7706
- Mean Squared Error: 132103731766.7099
- Mean Absolute Error: 103472.0474
- R2 Score: 0.8371
-----------------------------------
Model performance for Test set
- Root Mean Squared Error: 263888.0623
- Mean Squared Error: 69636909424.4648
- Mean Absolute Error: 117496.2131
- R2 Score: 0.9075


